# Generate conditions

In [23]:
import csv

import matplotlib.pyplot as plt


def make_conditioner(gender, age, bmi, glucose, height_cm, weight_kg, tc, tg, hdl, ldl, tc_hdl, hba1c):
  
    diabetes_status = evaluate_diabetes(float(glucose), hba1c, age, gender)
    cholesterol_status = is_cholesterol_normal(tc, ldl, hdl, tg, tc_hdl, gender)

    if diabetes_status and not cholesterol_status:
        dieses = "Diabetes"
    elif diabetes_status and cholesterol_status:
        dieses ="Cholesterol"
    elif cholesterol_status and not diabetes_status:
        dieses = "Cholesterol"
    else:
        dieses = None

    # bmi = calculate_bmi(weight_kg, height_cm)
    diet_recommendation = weight_recommendation(bmi)



    activity_level = 'moderately_active'

    if diet_recommendation == "Weight Loss":
        weight_change_per_week = 0.7 # pounds
    elif diet_recommendation == "Weight Gain":
        weight_change_per_week = 0.5
    else:
        weight_change_per_week =0

    daily_calories_needed = calculate_caloric_needs(weight_kg, height_cm, age, gender, activity_level, diet_recommendation, weight_change_per_week)
    # print(daily_calories_needed)
    if daily_calories_needed <1300:
        daily_need_calori = 1200
    elif daily_calories_needed >1300 and daily_calories_needed<1700:
        daily_need_calori = 1600
    elif daily_calories_needed>1701 and daily_calories_needed<2100:
        daily_need_calori = 2000
    elif daily_calories_needed>2101:
        daily_need_calori = 2400
    
    # print("daily_calories_needed",daily_calories_needed)
    return daily_need_calori, diet_recommendation, dieses


def calculate_caloric_needs(weight, height, age, gender, activity_level, goal='Maintain', weight_change_per_week=0):
    # BMR calculation based on gender
    if gender == 'male':
        bmr = 88.362 + (13.397 * weight) + (4.799 * height) - (5.677 * age)
    else:
        bmr = 447.593 + (9.247 * weight) + (3.098 * height) - (4.330 * age)
    
    # Activity multiplier
    activity_multipliers = {
        'sedentary': 1.2,
        'lightly_active': 1.375,
        'moderately_active': 1.55,
        'very_active': 1.725,
        'super_active': 1.9
    }
    
    tdee = bmr * activity_multipliers.get(activity_level, 1.2)  # Default to sedentary if no match
    
    # Caloric adjustment for weight change goal
    calorie_adjustment = weight_change_per_week * 500  # weight_change_per_week in pounds
    
    if goal == 'Weight Loss':
        daily_calories = tdee - calorie_adjustment
    elif goal == 'Weight Gain':
        daily_calories = tdee + calorie_adjustment
    else:
        daily_calories = tdee  # Maintain weight
    
    return daily_calories

def calculate_bmi(weight_kg, height_cm):
    # Convert height from cm to meters
    height_m = height_cm / 100
    
    # Calculate BMI
    bmi = weight_kg / (height_m ** 2)
    
    return round(bmi, 2)

def weight_recommendation(bmi):
    # Define BMI categories based on standard guidelines
    bmi_categories = {
        "underweight": (0, 18.5),
        "normal": (18.5, 24.9),
        "overweight": (25, 29.9),
        "obesity": (30, 39.9),
        "severe_obesity": (40, float('inf'))
    }
    # Determine the BMI category
    if bmi <18.5:
        category = "Weight Gain"
        return category
    elif bmi < 24.9:
        # maintain the same weight
        category = None
        return category
    elif bmi <29.9:
        category = "Weight Loss"
        return category
    elif bmi < 39.9:
        category = "Weight Loss"
        return category
    else:
        category = "Weight Loss"
    
    return category

def evaluate_diabetes(glucose, hba1c, age, gender):
    # Evaluate fasting glucose levels
    if glucose < 100:
        glucose_status = 'Normal'
    elif 100 <= glucose < 126:
        glucose_status = 'Prediabetes'
    else:
        glucose_status = 'Diabetes'
    
    # Evaluate HbA1c levels
    if hba1c < 5.7:
        hba1c_status = 'Normal'
    elif 5.7 <= hba1c < 6.5:
        hba1c_status = 'Prediabetes'
    else:
        hba1c_status = 'Diabetes'
    
    # Determine overall diabetes status
    if glucose_status == 'Diabetes' or hba1c_status == 'Diabetes':
        diabetes_status = True #'Diabetes'
    elif glucose_status == 'Prediabetes' or hba1c_status == 'Prediabetes':
        diabetes_status = True #'Prediabetes'
    else:
        diabetes_status = False #'Normal'
    
   

    return diabetes_status

def is_cholesterol_normal(tc, ldl, hdl, tg, tc_hdl_ratio, gender):
    # Total Cholesterol (Tc) Normal Range
    if not (tc < 200):
        return "Abnormal"
    
    # Low-Density Lipoprotein (LDL) Normal Range
    if not (ldl < 100):
        return "Abnormal"
    
    # High-Density Lipoprotein (HDL) Normal Range
    if gender == 'male':
        if not (hdl >= 40):
            return "Abnormal"
    elif gender == 'female':
        if not (hdl >= 50):
            return "Abnormal"
    
    # Triglycerides (Tg) Normal Range
    if not (tg < 150):
        return "Abnormal"
    
    # Total Cholesterol/HDL Ratio (Tc/Hdl) Normal Range
    if not (tc_hdl_ratio < 4):
        return "Abnormal"
    
    return "Normal"


## analyse data

In [25]:
# Specify the path to your CSV file
csv_file_path = "E:\SenzMate\Diet-Plan\Diet Recommendation\Diet Recommendation\query_dataset_dump.csv"

# Open the CSV file in read mode
with open(csv_file_path, 'r') as csvfile:
    reader = csv.DictReader(csvfile)

    # Skip the header row if it exists
    # next(reader, None)  # Assuming the header row is present

    cal_values =[]
    diet_recommendations = []
    # Read each row and extract values
    for row in reader:
        bmi = float(row['ï»¿bmi'])
        body_fat = float(row['body_fat'])
        glucose = float(row['glucose'])
        height = float(row['height'])
        weight = float(row['weight'])
        tc = float(row['tc'])
        tg = float(row['tg'])
        hdl = float(row['hdl'])
        ldl = float(row['ldl'])
        tc_hdl = float(row['tc_hdl'])
        bp_dias = float(row['bp_dias'])
        bp_sys = float(row['bp_sys'])
        hba1c = float(row['hba1c'])

        # Call the main function with the extracted values
        cal ,diet_recommendation,diabetes_status,cholesterol_status = main(gender, age, bmi, body_fat, glucose, height, weight, tc, tg, hdl, ldl, tc_hdl, bp_dias, bp_sys, hba1c)
    # Append the calculated 'cal' value to the list
        cal_values.append(cal)
        diet_recommendations.append(diet_recommendation)

    # Plot the distribution of 'cal' values (assuming they are numerical)
    plt.hist(cal_values)
    plt.xlabel("Calculated Value (cal)")
    plt.ylabel("Frequency")
    plt.title("Distribution of 'cal' Values")
    plt.show()

    plt.hist(diet_recommendations)
    plt.xlabel("diet_recommendations")
    plt.ylabel("Frequency")
    plt.title("Distribution of 'cal' Values")
    plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\SenzMate\\Diet-Plan\\Diet Recommendation\\Diet Recommendation\\query_dataset_dump.csv'

# KG data retrival

In [4]:
from neo4j import GraphDatabase

# generate a cyber text with conditions for KG data retrival.
uri = "neo4j+s://75b6527a.databases.neo4j.io"
user = "neo4j"
password = "dey-uWoTDE9TljwOirEmfnh6s0SpCeCb7GK4_oUUzRs"

driver = GraphDatabase.driver(uri, auth=(user, password))


def run_query(region, diet, condition, type, value):
    query = f"""
    MATCH (det:Diet_Details)-[s1:who_has]->(con:Condition)-[s2:aims]->(g:Goal)-[s3:can_have]->(b)-[s4:servings_of]->(bs)-[s5:contains_calories_of]->(bcal)
    WHERE det.Region = '{region}'
      AND det.Diet = '{diet}'
      AND con.Condition = '{condition}'
      AND g.Type = '{type}'
      AND det.Value = {value}
      
    RETURN
      det.Region AS Region,
      det.Diet AS Diet,
      det.Value AS Value,
      con.Condition AS Condition,
      g.Type AS GoalType,
      b AS BreakfastItems,
      bs AS BreakfastServings,
      bcal AS BreakfastCalories
    LIMIT 30
    """
    print(query)
    with driver.session() as session:
        result = session.run(query)
        print(result)
        return [record.data() for record in result]


Failed to write data to connection ResolvedIPv4Address(('34.126.64.110', 7687)) (ResolvedIPv4Address(('34.126.64.110', 7687)))
Failed to write data to connection IPv4Address(('75b6527a.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.64.110', 7687)))


In [5]:
from neo4j import GraphDatabase

In [6]:
uri = "neo4j+s://75b6527a.databases.neo4j.io"
user = "neo4j"
password = "dey-uWoTDE9TljwOirEmfnh6s0SpCeCb7GK4_oUUzRs"

driver = GraphDatabase.driver(uri, auth=(user, password))

In [7]:
# Define the query
query_break = """
MATCH (ft)-[:foods_for]->(dis)-[:condition_who_aims]->(g)-[:and_likes]->(fc)-[:can_have]->(b:Breakfast_Items)
WHERE fc.Region = '{region}'
  AND ft.Diet = '{diet}'
  AND g.Type = '{d_type}'
  AND dis.Condition = '{condition}'
RETURN DISTINCT b.br as Breakfast_combo
LIMIT 4
"""
query_dinner = """
MATCH (ft)-[:foods_for]->(dis)-[:condition_who_aims]->(g)-[:and_likes]->(fc)-[:can_have]->(d:Dinner_Items) 
WHERE fc.Region = '{region}'
  AND ft.Diet = '{diet}'
  AND g.Type = '{d_type}'
  AND dis.Condition = '{condition}'
RETURN DISTINCT d.dn as Dinner_combo
LIMIT 4
"""

query_lunch = """
MATCH (ft)-[:foods_for]->(dis)-[:condition_who_aims]->(g)-[:and_likes]->(fc)-[:can_have]->(l:Lunch_Items) 
WHERE fc.Region = '{region}'
  AND ft.Diet = '{diet}'
  AND g.Type = '{d_type}'
  AND dis.Condition = '{condition}'
RETURN DISTINCT l.lu as Lunch_combo
LIMIT 3
"""

query_morn_snack = """
MATCH (ft)-[:foods_for]->(dis)-[:condition_who_aims]->(g)-[:and_likes]->(fc)-[:can_have]->(m:Mor_snac_Items) 
WHERE fc.Region = '{region}'
  AND ft.Diet = '{diet}'
  AND g.Type = '{d_type}'
  AND dis.Condition = '{condition}'
RETURN DISTINCT m.ms as Morning_snack_combo
LIMIT 2
"""

query_post_work = """
MATCH (ft)-[:foods_for]->(dis)-[:condition_who_aims]->(g)-[:and_likes]->(fc)-[:can_have]->(po:Post_workout_snack_Items) 
WHERE fc.Region = '{region}'
  AND ft.Diet = '{diet}'
  AND g.Type = '{d_type}'
  AND dis.Condition = '{condition}'
RETURN DISTINCT po.pos as Post_workout_snack_combo
LIMIT 3
"""
query_pre_work = """
MATCH (ft)-[:foods_for]->(dis)-[:condition_who_aims]->(g)-[:and_likes]->(fc)-[:can_have]->(pr:Pre_workout_snack_Items) 
WHERE fc.Region = '{region}'
  AND ft.Diet = '{diet}'
  AND g.Type = '{d_type}'
  AND dis.Condition = '{condition}'
RETURN DISTINCT pr.prs as Post_workout_snack_combo
LIMIT 2
"""
def KG_data_retiver(region,diet,d_type,condition):
  kg_data = []
  querys = [query_break,query_lunch,query_dinner,query_morn_snack,query_post_work,query_pre_work]
  for query in querys:
    food_query =  query.format(region=region,diet=diet,d_type=d_type,condition=condition)
    # Execute the query and collect the results
    with driver.session() as session:
        results = session.run(food_query)
        food_items = [record for record in results]
    kg_data.append(food_items)
    # # Print the collected results
    # for result in food_items:
    #   print(result)

  return kg_data


kg_data = KG_data_retiver('South Indian','Non Veg','Weight Gain','Diabetes')
print(kg_data)

[[<Record Breakfast_combo='Upma'>, <Record Breakfast_combo='Tomato peanut chutney | Green Gram Idli'>, <Record Breakfast_combo='Paneer bhurji dosa | Drumstick sambhar'>, <Record Breakfast_combo='Buttermilk | Vermicelli upma'>], [<Record Lunch_combo='Carrot cucumber salad | Daal | Tofu | Brown rice'>, <Record Lunch_combo='Ragi/nachni roti | Green salad | Coconut chickpea curry | Methi Bhaji'>, <Record Lunch_combo='Tomato cucumber raita | Soya chunk pulao'>], [<Record Dinner_combo='Wheat Roti | Gobi matar sabzi | Daal'>, <Record Dinner_combo='Brinjal sambhar | Bean salad | Brown rice'>, <Record Dinner_combo='Wheat Roti | Daal | Mix Veg Bhaji'>, <Record Dinner_combo='Chapati | Bhindi Sabzi | Daal | Carrot tomato salad'>], [<Record Morning_snack_combo='Apple | Cashew Nuts | Almonds'>, <Record Morning_snack_combo='Coconut water | Almonds | Cashew Nuts'>], [<Record Post_workout_snack_combo='Moong beans boiled'>, <Record Post_workout_snack_combo='Epigamia plain yoghurt'>, <Record Post_workout

In [8]:
import csv
import json
def csv_to_jsonl_selected_columns(csv_file_path, jsonl_file_path, columns):
    with open(csv_file_path, mode='r', encoding='utf-8') as csv_file, \
         open(jsonl_file_path, mode='w', encoding='utf-8') as jsonl_file:
        
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            filtered_row = {col: row[col] for col in columns if col in row}
            jsonl_file.write(json.dumps(filtered_row) + '\n')

# Example usage
csv_file_path = 'E:\\SenzMate\\Diet-Plan\\all_food.csv'
jsonl_file_path = 'output_file.jsonl'
columns = ['Item', 'Serving','Calories']  # Replace with the columns you want to include
csv_to_jsonl_selected_columns(csv_file_path, jsonl_file_path, columns)


In [9]:
def load_jsonl_as_string(jsonl_file_path):
    with open(jsonl_file_path, 'r', encoding='utf-8') as jsonl_file:
        return jsonl_file.read()

# Example usage
jsonl_file_path = 'output_file.jsonl'
jsonl_data = load_jsonl_as_string(jsonl_file_path)
print(jsonl_data)

{"Item": "Upma", "Serving": "1.5 katori", "Calories": "256.5"}
{"Item": "Tomato peanut chutney", "Serving": "2 tbsp", "Calories": "30"}
{"Item": "Paneer bhurji dosa", "Serving": "1 dosa", "Calories": "369"}
{"Item": "Buttermilk", "Serving": "1 glass", "Calories": "45"}
{"Item": "Paneer Burji", "Serving": "1 Small", "Calories": "209"}
{"Item": "Dahi", "Serving": "1 katori", "Calories": "90"}
{"Item": "Peas poha", "Serving": "1 katori", "Calories": "140"}
{"Item": "Egg Omelette", "Serving": "2 Piece", "Calories": "208"}
{"Item": "Paneer Paratha", "Serving": "2 Piece", "Calories": "448"}
{"Item": "Boiled Egg", "Serving": "2 Egg", "Calories": "172"}
{"Item": "Wheat Roti", "Serving": "2 roti", "Calories": "170"}
{"Item": "Oats rava idli", "Serving": "4 piece", "Calories": "296"}
{"Item": "Chana Soyabean Bhaji", "Serving": "1 Katori", "Calories": "141"}
{"Item": "Coconut Chutney", "Serving": "4 Teaspoon", "Calories": "68"}
{"Item": "Poha", "Serving": "1 Katori", "Calories": "130"}
{"Item": "

# RAG LLM

In [10]:
import json
import os

import google.generativeai as genai
from google.generativeai.types import HarmBlockThreshold, HarmCategory
from langchain_google_genai import ChatGoogleGenerativeAI


class GeminiInitializer:
    def __init__(self, config_path="E:\SenzMate\Diet-Plan\Code\config.json"):
        with open(config_path) as f:
            config = json.load(f)
        self.api_key = config["gemini-api-key"]
        os.environ['GOOGLE_API_KEY'] = self.api_key
        genai.configure(api_key = os.environ['GOOGLE_API_KEY'])
        print("Initializing the model")

    def run_text_model(self,
        prompt: str,
        model_name: str = "gemini-1.0-pro",
        temperature: float = 0
        ):
        safety_settings={HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_NONE,
                    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_NONE}
        llm = ChatGoogleGenerativeAI(model=model_name, temperature=temperature, safety_settings=safety_settings)
        response = llm.invoke(prompt)
        return response.content

    def extract_entities_relationships(self, prompt, model_name):
        try:
            res = self.run_text_model(prompt=prompt, model_name = model_name, temperature = 0)
            return res
        except Exception as e:
            print(e)


c:\Users\sajee\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [26]:
import json

# from gemini_initializer import GeminiInitializer

base_prompt = """
You are a diet recommendation AI bot that provides personalized meal plans. 
Your primary job is to generate a detailed meal plans for the seven days. The meal plan should include breakfast, lunch, dinner, morning snacks, post-workout snacks, and pre-workout snacks with respective serving quantities.

<INSTRUCTIONS>
1. Please comprehensively read the given documents for the relevent_food_combo, food_item_data and provided constraints (user_calorie_needs) to get proper understanding. 
2. Remember your primary job is to generate a detailed meal plans for sseven days. The meal plan should include breakfast, lunch, dinner, morning snacks, post-workout snacks, and pre-workout snacks with respective serving quantities.
3. Based on the user's relevent_food_combo, food_item_data generate a detailed 1-day meal plan for seven days. Don't suggest same food combination for all the days.
4. Ensure that the meal plan includes appropriate recommendations for breakfast, lunch, dinner, morning snacks, evening snacks, post-workout snacks, and pre-workout snacks
with respective serving quantities.
5. Generate meal plan must be aligned with user's health conditions and dietary preferences. Here health condition should prioritize over dietary preferences by the user.
6. Ensure that the meals are nutritionally balanced and appropriate for the user's health conditions.Optimize the results with the most appropriate diet plan. 

</INSTRUCTIONS>

<Constraints>
1.If you cannot find any information on the entities and relationships in match_kg_data, return the string 'N/A'.
2.Avoid suggesting any fictitious data.
3.You must not provide duplicate entities in your final answer.
4.DO NOT miss out on any information.
5.DO NOT impute any missing values.
6.STRICTLY NEED the output in JSON format include day as main key as given in the example which is for 3 days meal plan.
</Constraints>

<Example>
Exapmle for three day meal plan.
{
    "Day-1": {
        "Breakfast": {
            "Item": "Upma",
            "Serving Quantity": "200g"
        },
        "Morning Snack": {
            "Item": "Apple",
            "Serving Quantity": "1 medium"
        },
        "Lunch": {
            "Item": "2 Wheat Roti, Daal, Gobi matar sabzi, Carrot cucumber salad",
            "Serving Quantity": "2, 1 cup, 1 cup, 1 cup"
        },
    
        "Dinner": {
            "Item": "2 Wheat Roti, Daal, Tofu",
            "Serving Quantity": "2, 1 cup, 100 gms"
        },
        "Pre-workout Snack": {
            "Item": "Papaya",
            "Serving Quantity": "1 cup"
        },
        "Post-workout Snack": {
            "Item": "Moong beans boiled",
            "Serving Quantity": "1/2 cup"
        }
    },
    "Day-2": {
        "Breakfast": {
            "Item": "Upma",
            "Serving Quantity": "200g"
        },
        "Morning Snack": {
            "Item": "Cashew Nuts",
            "Serving Quantity": "10 gms"
        },
        "Lunch": {
            "Item": "2 Wheat Roti, Daal, Gobi matar sabzi, Carrot cucumber salad",
            "Serving Quantity": "2, 1 cup, 1 cup, 1 cup"
        },
   
        "Dinner": {
            "Item": "2 Wheat Roti, Daal, Tofu",
            "Serving Quantity": "2, 1 cup, 100 gms"
        },
        "Pre-workout Snack": {
            "Item": "Chia seeds",
            "Serving Quantity": "1 tbsp"
        },
        "Post-workout Snack": {
            "Item": "Moong beans boiled",
            "Serving Quantity": "1/2 cup"
        }
    },
    "Day-3": {
        "Breakfast": {
            "Item": "Upma",
            "Serving Quantity": "200g"
        },
        "Morning Snack": {
            "Item": "Almonds",
            "Serving Quantity": "10 gms"
        },
        "Lunch": {
            "Item": "2 Wheat Roti, Daal, Gobi matar sabzi, Carrot cucumber salad",
            "Serving Quantity": "2, 1 cup, 1 cup, 1 cup"
        },
     
        "Dinner": {
            "Item": "2 Wheat Roti, Daal, Tofu",
            "Serving Quantity": "2, 1 cup, 100 gms"
        },
        "Pre-workout Snack": {
            "Item": "Pear",
            "Serving Quantity": "1 medium"
        },
        "Post-workout Snack": {
            "Item": "Moong beans boiled",
            "Serving Quantity": "1/2 cup"
        }
    }
}

</Example>
"""
template = """

<DOCUMENT>
user_calorie_needs:{user_calorie_needs}
relevent_food_combo:{relevent_food_combo}
food_item_data: {food_item_data}
</DOCUMENT>

Answer:

"""

def gemini_bot(user_calorie_needs, relevent_food_combo,food_item_data, gemini=GeminiInitializer()):
    """
    Enhances resident care plan notes by correcting grammar, spelling, and suggesting paraphrases to improve clarity and conciseness.

    Args:
    - text (str): Input text containing a care plan for a resident.
    - gemini (GeminiInitializer): Instance of GeminiInitializer class.

    Returns:
    - str: Improved version of the input text.
    """
    prompt = base_prompt + template.format(user_calorie_needs=user_calorie_needs,relevent_food_combo=relevent_food_combo,food_item_data=food_item_data)
    # print(prompt)
    response = gemini.run_text_model(prompt, model_name="gemini-1.5-pro-latest", temperature=0.2)
    print(response)

    return response



Initializing the model


In [27]:
user_calorie_needs =2000
response = gemini_bot(user_calorie_needs, kg_data,jsonl_data)

```json
{
    "Day-1": {
        "Breakfast": {
            "Item": "Upma",
            "Serving Quantity": "1.5 katori"
        },
        "Morning Snack": {
            "Item": "Apple",
            "Serving Quantity": "1 piece"
        },
        "Lunch": {
            "Item": "Carrot cucumber salad, Daal, Tofu, Brown rice",
            "Serving Quantity": "1 katori, 1 bowl, 1 serving, 1 katori"
        },
        "Dinner": {
            "Item": "Wheat Roti, Gobi matar sabzi, Daal",
            "Serving Quantity": "2 roti, 1 katori, 1 bowl"
        },
        "Pre-workout Snack": {
            "Item": "Papaya",
            "Serving Quantity": "1 cup"
        },
        "Post-workout Snack": {
            "Item": "Moong beans boiled",
            "Serving Quantity": "1 katori"
        }
    },
    "Day-2": {
        "Breakfast": {
            "Item": "Tomato peanut chutney | Green Gram Idli",
            "Serving Quantity": "2 tbsp, 3 Piece"
        },
        "Morning Snack": {
     

In [28]:
# response = diet_planner.gemini_bot(daily_need_calori,kg_data)
cleaned_data = response.replace('```json', '').replace('```', '').strip()

diet_plan = json.loads(cleaned_data)

# Create an empty list to store the data
data = []
meals_time = ["Day","Breakfast","Morning Snack", "Lunch","Dinner","Pre-workout Snack","Post-workout Snack"]

# # Iterate over each day in the JSON data
# for day, meals in diet_plan.items():
#     meals_time.append(meals)
data.append(meals_time)
for day, meals in diet_plan.items():
    row_data = [day]
    for meal_time, meal_details in meals.items():
        # Extract item and serving quantity
        item = meal_details["Item"]
        serving_quantity = meal_details["Serving Quantity"]

        json_string = f"Item :{item} \n Serving Quantity :{serving_quantity}"
        row_data.append(json_string)
    data.append(row_data)



In [29]:
# Create a DataFrame
import pandas as pd
df = pd.DataFrame(data)

In [30]:
df

,0,1,2,3,4,5,6
0,Day,Breakfast,Morning Snack,Lunch,Dinner,Pre-workout Snack,Post-workout Snack
1,Day-1,Item :Upma \n Serving Quantity :1.5 katori,Item :Apple \n Serving Quantity :1 piece,"Item :Carrot cucumber salad, Daal, Tofu, Brown...","Item :Wheat Roti, Gobi matar sabzi, Daal \n Se...",Item :Papaya \n Serving Quantity :1 cup,Item :Moong beans boiled \n Serving Quantity :...
2,Day-2,Item :Tomato peanut chutney | Green Gram Idli ...,Item :Cashew Nuts \n Serving Quantity :4 Piece,"Item :Ragi/nachni roti, Green salad, Coconut c...","Item :Brinjal sambhar, Bean salad, Brown rice ...",Item :Chia seeds \n Serving Quantity :1 tbsp,Item :Epigamia plain yoghurt \n Serving Quanti...
3,Day-3,Item :Paneer bhurji dosa | Drumstick sambhar \...,Item :Almonds \n Serving Quantity :4 almond,"Item :Tomato cucumber raita, Soya chunk pulao ...","Item :Wheat Roti, Daal, Mix Veg Bhaji \n Servi...",Item :Mosambi \n Serving Quantity :1 piece,Item :Boiled moong and chana \n Serving Quanti...
4,Day-4,"Item :Buttermilk, Vermicelli upma \n Serving Q...",Item :Coconut water \n Serving Quantity :1 piece,"Item :Carrot cucumber salad, Daal, Tofu, Brown...","Item :Chapati, Bhindi Sabzi, Daal, Carrot toma...",Item :Papaya \n Serving Quantity :1 cup,Item :Moong beans boiled \n Serving Quantity :...
5,Day-5,Item :Upma \n Serving Quantity :1.5 katori,Item :Apple \n Serving Quantity :1 piece,"Item :Ragi/nachni roti, Green salad, Coconut c...","Item :Wheat Roti, Gobi matar sabzi, Daal \n Se...",Item :Chia seeds \n Serving Quantity :1 tbsp,Item :Epigamia plain yoghurt \n Serving Quanti...
6,Day-6,Item :Tomato peanut chutney | Green Gram Idli ...,Item :Cashew Nuts \n Serving Quantity :4 Piece,"Item :Tomato cucumber raita, Soya chunk pulao ...","Item :Brinjal sambhar, Bean salad, Brown rice ...",Item :Mosambi \n Serving Quantity :1 piece,Item :Boiled moong and chana \n Serving Quanti...
7,Day-7,Item :Paneer bhurji dosa | Drumstick sambhar \...,Item :Almonds \n Serving Quantity :4 almond,"Item :Carrot cucumber salad, Daal, Tofu, Brown...","Item :Chapati, Bhindi Sabzi, Daal, Carrot toma...",Item :Papaya \n Serving Quantity :1 cup,Item :Moong beans boiled \n Serving Quantity :...


# Diet Planner

In [33]:


# common details
gender = "Male"
age = 70
food_preference = "Veg"
food_cousin = "South Indian"
height_cm = 164
weight_kg = 70
bmi =295

# Medical details
body_Fat = 19.71
glucose = 245
tc = 166
tg = 175
hdl = 55
ldl  = 97
tc_hdl = 3.77
bp_dias = 113
bp_sys = 66
hba1c = 2.2
# print(main(gender, age, bmi, glucose, height_cm, weight_kg, tc, tg, hdl, ldl, tc_hdl, bp_dias, bp_sys, hba1c))

In [34]:
daily_need_calori, diet_recommendation, dieses = make_conditioner(gender, age, 
                                                                                             bmi, 
                                                                                             glucose, 
                                                                                            height_cm, 
                                                                                            weight_kg, 
                                                                                            tc, 
                                                                                            tg, 
                                                                                            hdl, 
                                                                                            ldl, 
                                                                                            tc_hdl, 
                                                                                            hba1c)
kg_data = run_query(food_cousin, 
                    food_preference, 
                    dieses, 
                    diet_recommendation, 
                    daily_need_calori)
data = {
        "food_type": food_cousin,
        "food_preference": food_preference,
        "diseases": dieses,
        "diet_recommendation": diet_recommendation,
        "daily_need_calories": daily_need_calori
            }
print(kg_data)
response = gemini_bot(data,kg_data)
# print(response)


    MATCH (det:Diet_Details)-[s1:who_has]->(con:Condition)-[s2:aims]->(g:Goal)-[s3:can_have]->(b)-[s4:servings_of]->(bs)-[s5:contains_calories_of]->(bcal)
    WHERE det.Region = 'South Indian'
      AND det.Diet = 'Veg'
      AND con.Condition = 'Cholesterol'
      AND g.Type = 'Weight Loss'
      AND det.Value = 1600
      
    RETURN
      det.Region AS Region,
      det.Diet AS Diet,
      det.Value AS Value,
      con.Condition AS Condition,
      g.Type AS GoalType,
      b AS BreakfastItems,
      bs AS BreakfastServings,
      bcal AS BreakfastCalories
    LIMIT 30
    
[{'Region': 'South Indian', 'Diet': 'Veg', 'Value': 1600, 'Condition': 'Cholesterol', 'GoalType': 'Weight Loss', 'BreakfastItems': {'b2': 'NaN', 'b3': 'NaN', 'b4': 'NaN', 'b1': 'Upma'}, 'BreakfastServings': {'bs2': 'NaN', 'bs1': '1.5 katori', 'bs4': 'NaN', 'bs3': 'NaN'}, 'BreakfastCalories': {'bcal3': 'NaN', 'bcal2': 'NaN', 'bcal1': 256.5, 'bcal4': 'NaN'}}, {'Region': 'South Indian', 'Diet': 'Veg', 'Value': 1600

In [2]:
import re

data = '''{
    "Day-1": {
        "Breakfast": {
            "Item": "Upma",
            "Serving Quantity": "1 cup"
        },
        "Morning Snack": {
            "Item": "Apple",
            "Serving Quantity": "1 medium"
        },
        "Lunch": {
            "Item": "2 Wheat Roti, Daal, Gobi matar sabzi, Carrot cucumber salad",
            "Serving Quantity": "2, 1 cup, 1 cup, 1 cup"
        },
        "Evening Snack": {
            "Item": "Roasted chana",
            "Serving Quantity": "1/2 cup"
        },
        "Dinner": {
            "Item": "2 Wheat Roti, Daal, Tofu",
            "Serving Quantity": "2, 1 cup, 100 gms"
        },
        "Pre-workout Snack": {
            "Item": "Papaya",
            "Serving Quantity": "1 cup"
        },
        "Post-workout Snack": {
            "Item": "Moong beans boiled",
            "Serving Quantity": "1/2 cup"
        }
    },
    "Day-2": {
        "Breakfast": {
            "Item": "Upma",
            "Serving Quantity": "1 cup"
        },
        "Morning Snack": {
            "Item": "Cashew Nuts",
            "Serving Quantity": "10 gms"
        },
        "Lunch": {
            "Item": "2 Wheat Roti, Daal, Gobi matar sabzi, Carrot cucumber salad",
            "Serving Quantity": "2, 1 cup, 1 cup, 1 cup"
        },
        "Evening Snack": {
            "Item": "Roasted chana",
            "Serving Quantity": "1/2 cup"
        },
        "Dinner": {
            "Item": "2 Wheat Roti, Daal, Tofu",
            "Serving Quantity": "2, 1 cup, 100 gms"
        },
        "Pre-workout Snack": {
            "Item": "Chia seeds",
            "Serving Quantity": "1 tbsp"
        },
        "Post-workout Snack": {
            "Item": "Moong beans boiled",
            "Serving Quantity": "1/2 cup"
        }
    },
    "Day-3": {
        "Breakfast": {
            "Item": "Upma",
            "Serving Quantity": "1 cup"
        },
        "Morning Snack": {
            "Item": "Almonds",
            "Serving Quantity": "10 gms"
        },
        "Lunch": {
            "Item": "2 Wheat Roti, Daal, Gobi matar sabzi, Carrot cucumber salad",
            "Serving Quantity": "2, 1 cup, 1 cup, 1 cup"
        },
        "Evening Snack": {
            "Item": "Roasted chana",
            "Serving Quantity": "1/2 cup"
        },
        "Dinner": {
            "Item": "2 Wheat Roti, Daal, Tofu",
            "Serving Quantity": "2, 1 cup, 100 gms"
        },
        "Pre-workout Snack": {
            "Item": "Pear",
            "Serving Quantity": "1 medium"
        },
        "Post-workout Snack": {
            "Item": "Moong beans boiled",
            "Serving Quantity": "1/2 cup"
        }
    },
    "Day-4": {
        "Breakfast": {
            "Item": "Upma",
            "Serving Quantity": "1 cup"
        },
        "Morning Snack": {
            "Item": "Apple",
            "Serving Quantity": "1 medium"
        },
        "Lunch": {
            "Item": "2 Wheat Roti, Daal, Gobi matar sabzi, Carrot cucumber salad",
            "Serving Quantity": "2, 1 cup, 1 cup, 1 cup"
        },
        "Evening Snack": {
            "Item": "Roasted chana",
            "Serving Quantity": "1/2 cup"
        },
        "Dinner": {
            "Item": "2 Wheat Roti, Daal, Tofu",
            "Serving Quantity": "2, 1 cup, 100 gms"
        },
        "Pre-workout Snack": {
            "Item": "Granola bar",
            "Serving Quantity": "1"
        },
        "Post-workout Snack": {
            "Item": "Moong beans boiled",
            "Serving Quantity": "1/2 cup"
        }
    },
    "Day-5": {
        "Breakfast": {
            "Item": "Upma",
            "Serving Quantity": "1 cup"
        },
        "Morning Snack": {
            "Item": "Cashew Nuts",
            "Serving Quantity": "10 gms"
        },
        "Lunch": {
            "Item": "2 Wheat Roti, Daal, Gobi matar sabzi, Carrot cucumber salad",
            "Serving Quantity": "2, 1 cup, 1 cup, 1 cup"
        },
        "Evening Snack": {
            "Item": "Roasted chana",
            "Serving Quantity": "1/2 cup"
        },
        "Dinner": {
            "Item": "2 Wheat Roti, Daal, Tofu",
            "Serving Quantity": "2, 1 cup, 100 gms"
        },
        "Pre-workout Snack": {
            "Item": "Apple",
            "Serving Quantity": "1 medium"
        },
        "Post-workout Snack": {
            "Item": "Moong beans boiled",
            "Serving Quantity": "1/2 cup"
        }
    },
    "Day-6": {
        "Breakfast": {
            "Item": "Upma",
            "Serving Quantity": "1 cup"
        },
        "Morning Snack": {
            "Item": "Almonds",
            "Serving Quantity": "10 gms"
        },
        "Lunch": {
            "Item": "2 Wheat Roti, Daal, Gobi matar sabzi, Carrot cucumber salad",
            "Serving Quantity": "2, 1 cup, 1 cup, 1 cup"
        },
        "Evening Snack": {
            "Item": "Roasted chana",
            "Serving Quantity": "1/2 cup"
        },
        "Dinner": {
            "Item": "2 Wheat Roti, Daal, Tofu",
            "Serving Quantity": "2, 1 cup, 100 gms"
        },
        "Pre-workout Snack": {
            "Item": "Cashew Nuts",
            "Serving Quantity": "10 gms"
        },
        "Post-workout Snack": {
            "Item": "Moong beans boiled",
            "Serving Quantity": "1/2 cup"
        }
    },
    "Day-7": {
        "Breakfast": {
            "Item": "Upma",
            "Serving Quantity": "1 cup"
        },
        "Morning Snack": {
            "Item": "Apple",
            "Serving Quantity": "1 medium"
        },
        "Lunch": {
            "Item": "2 Wheat Roti, Daal, Gobi matar sabzi, Carrot cucumber salad",
            "Serving Quantity": "2, 1 cup, 1 cup, 1 cup"
        },
        "Evening Snack": {
            "Item": "Roasted chana",
            "Serving Quantity": "1/2 cup"
        },
        "Dinner": {
            "Item": "2 Wheat Roti, Daal, Tofu",
            "Serving Quantity": "2, 1 cup, 100 gms"
        },
        "Pre-workout Snack": {
            "Item": "Almonds",
            "Serving Quantity": "10 gms"
        },
        "Post-workout Snack": {
            "Item": "Moong beans boiled",
            "Serving Quantity": "1/2 cup"
        }
    }
}'''

from langchain_core.output_parsers import JsonOutputParser

parser = JsonOutputParser()
print(parser(data))


TypeError: 'JsonOutputParser' object is not callable